In [1]:
import numpy as np
import pandas as pd
import datatable
import itertools as it
from owlready2 import *
import math
import joblib
import copy
import pickle
from sklearn.feature_selection import mutual_info_classif
from collections import Counter
#para depurar los que letras que no están en el codigo ascii
import unicodedata
import functools
import spacy
import stanfordnlp
#from spacy_stanfordnlp import StanfordNLPLanguage
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import metrics
#from wiki_dump_reader import Cleaner, iterate
from sklearn.decomposition import TruncatedSVD

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize


#from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
#from gensim.models.wrappers import LdaMallet
#from gensim.corpora import Dictionary

#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import SVG

import random
from sklearn.externals import joblib


#Operaciones con gráficos
import scattertext as st
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

import networkx as nx

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool, LabelSet, ColumnDataSource, Range1d
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE

#import gensim
#import gensim.corpora as corpora
#from gensim.models import CoherenceModel
#from gensim.models import HdpModel


import spacy
import time 
from spacy_langdetect import LanguageDetector
from owlready2 import *
import json 
from spotlight import *
import nltk
from nltk.corpus import wordnet as wn



warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from multiprocessing import  Pool
import math
import scipy.sparse as sp

import vectorizer 
importlib.reload(vectorizer)
import rulesets

importlib.reload(rulesets)

from vectorizer import  WeightedTfidfVectorizer
from rulesets import *

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from owlready2 import *

from mdutils.mdutils import MdUtils
from mdutils import Html

import rdflib
import joblib
import h2o

# Carga de datos

In [2]:

dtm_scaled = pd.read_csv('../data/interim/amazon/weighteddtmscaled.csv',sep=',')
del dtm_scaled['Unnamed: 0']
dtm_scaled.head()
dtm_scaled = dtm_scaled.astype('float32')
cut_labels_4 = [1,2,3,4,5,6,7,8,9,10]
for column in dtm_scaled.columns:
    if column=='target':
        continue
    dtm_scaled[column]  = pd.cut(dtm_scaled[column], bins=10, labels=cut_labels_4)

dtm_scaled.head()


,word,romance,definitely,young,star,follow,bite,begin,line,amaze,...,month,phone,computer,charge,ship,device,usb,right,doesn,target
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1.0
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1.0
2,1,1,5,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1.0
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1.0
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1.0


In [3]:
# Topics e importancia de variables

In [4]:
## 1 Experimento: Número de reglas: 10, usando un máximo de 10 términos, sin selección de vocabulario
pds = copy.deepcopy(dtm_scaled)

In [5]:
rs = RuleSet()
rs.target='target'
rs.pds=pds
rs.pds_original = pds 
explainer = Explainer(rs,1,10,10,200)
rs, pdsb = explainer.explain()

*****************************Comienza la genreación del ruleset************************************************
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: book
Igualdad generada: book==3POSITIVOS 4644.0NEGATIVOS: 255
Igualdad de la regla book==3
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: read
Igualdad generada: read==7POSITIVOS 34.0NEGATIVOS: 0
Igualdad de la regla read==7
-----------------> COUNTER USADOS
Counter({'book': 1, 'read': 1})
|2 - Regla generada: |book==3 and read==7 => p(0.0), conf(0.0012440086348834657) 
ruleset query(book==3 and read==7) 
	 |2.1 -Quedan | 27297 positivos, generadas 2 reglas
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: book
Igualdad generada: book==3POSITIVOS 4610.0NEGATIVOS: 255
Igualdad de la regla book==3
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: read
Igualdad generada: read==9POSITIVOS 2.0NEGATIVOS: 0
Igualdad de la regla read==9


In [6]:
rso = copy.deepcopy(rs)
for key in rso.dictionary.keys():
    print(rso.dictionary.get(key).toString())

book==3 and read==7 => p(0.0), conf(0.0012440086348834657)
book==3 and read==9 => p(0.0), conf(7.31769785225568e-05)
book==3 and read==8 => p(0.0), conf(0.0005854158281804544)
book==3 and read==6 and story==7 => p(0.0), conf(7.31769785225568e-05)
book==3 and read==6 and story==4 => p(0.0), conf(0.00025611942482894884)
book==3 and read==6 and story==3 => p(0.0), conf(0.000182942446306392)
book==3 and read==6 and story==5 => p(0.0), conf(3.65884892612784e-05)
book==3 and read==6 and story==2 => p(0.0), conf(7.31769785225568e-05)
book==3 and read==6 and story==1 and character==2 => p(0.0), conf(3.65884892612784e-05)
book==3 and read==6 and story==1 and character==3 => p(0.0), conf(0.0001097654677838352)


In [7]:
consolidar = rso.consolidar()

0
diccionary
{0: <rulesets.Rule object at 0x7faf55e17090>, 1: <rulesets.Rule object at 0x7faf55e17d10>, 2: <rulesets.Rule object at 0x7faf55e17950>, 3: <rulesets.Rule object at 0x7faf55e17190>, 4: <rulesets.Rule object at 0x7faf55e17710>, 5: <rulesets.Rule object at 0x7faf55e17750>, 6: <rulesets.Rule object at 0x7faf55e3cc50>, 7: <rulesets.Rule object at 0x7faf55e3c290>, 8: <rulesets.Rule object at 0x7faf55e3ced0>, 9: <rulesets.Rule object at 0x7faf55e3c8d0>}
visitados q no deben volver :0 -- 0 -- 1 -- 2
listado de reglas con mismo antecedente:
book==3 and read==7 => p(0.0), conf(0.0012440086348834657)
book==3 and read==9 => p(0.0), conf(7.31769785225568e-05)
book==3 and read==8 => p(0.0), conf(0.0005854158281804544)
listado_vari::
book read
  book read  confianza  consecuente
0    3    7   0.001244          0.0
2    3    8   0.000585          0.0
1    3    9   0.000073          0.0
book==3 and read==7 => p(0.0), conf(0.0012440086348834657)
book==3 and read==8 => p(0.0), conf(0.0005854

In [8]:
contador = 1
for ley in consolidar.keys():
    print(consolidar.get(ley).toString())
    joblib.dump(consolidar.get(ley),'../objects/amazon/rules_1_experiment/rule_1_'+str(contador))
    contador = contador+1

debería entrar aqui si tengo antecedentes, por cada disyuncion
OPERATOR GREATER OR LOWER THAN
operator 1>=
book==3 and ((read>=7 and read<=9) or 
book==3 and ((read>=7 and read<=9)) => p(0.0), conf(0.0019026014415864769)
debería entrar aqui si tengo antecedentes, por cada disyuncion
OPERATOR IGUAL story==5--story==5
book==3 and read==6 and (story==5 or 
book==3 and read==6 and (story==5 or 
debería entrar aqui si tengo antecedentes, por cada disyuncion
OPERATOR IGUAL story==7--story==7
book==3 and read==6 and (story==5 or story==7 or 
book==3 and read==6 and (story==5 or story==7 or 
book==3 and read==6 and (story==5 or story==7) => p(0.0), conf(0.0001097654677838352)
debería entrar aqui si tengo antecedentes, por cada disyuncion
OPERATOR GREATER OR LOWER THAN
operator 1>=
book==3 and read==6 and ((story>=3 and story<=4) or 
book==3 and read==6 and ((story>=3 and story<=4)) => p(0.0), conf(0.0004390618711353408)
book==3 and read==6 and story==2 => p(0.0), conf(7.31769785225568e-05)
deb

In [25]:
## 2 Experimento: Número de reglas 15, selección de vocabulario
vocabulary=['word','book','romance','story','author','character','read']
rs = RuleSet()
rs.target='target'
rs.pds=copy.deepcopy(dtm_scaled)
rs.pds_original = copy.deepcopy(dtm_scaled) 
explainer = Explainer(rs,1,10,15,200,vocabulary=vocabulary)
rs, pdsb = explainer.explain()


*****************************Comienza la genreación del ruleset************************************************
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: book
Igualdad generada: book==8POSITIVOS 23.0NEGATIVOS: 4
Igualdad de la regla book==8
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: read
Igualdad generada: read==5POSITIVOS 13.0NEGATIVOS: 0
Igualdad de la regla read==5
-----------------> COUNTER USADOS
Counter({'book': 1, 'read': 1})
|2 - Regla generada: |book==8 and read==5 => p(0.0), conf(0.0004756503603966192) 
ruleset query(book==8 and read==5) 
	 |2.1 -Quedan | 27318 positivos, generadas 2 reglas
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: book
Igualdad generada: book==4POSITIVOS 23.0NEGATIVOS: 4
Igualdad de la regla book==4
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: read
Igualdad generada: read==10POSITIVOS 4.0NEGATIVOS: 0
Igualdad de la regla read==10
------

In [26]:
rso = copy.deepcopy(rs)
for key in rso.dictionary.keys():
    print(rso.dictionary.get(key).toString())

book==8 and read==5 => p(0.0), conf(0.0004756503603966192)
book==4 and read==10 => p(0.0), conf(0.0001463539570451136)
book==4 and read==7 => p(0.0), conf(0.0005854158281804544)
book==4 and read==8 => p(0.0), conf(0.0004756503603966192)
book==4 and read==6 => p(0.0), conf(0.001353774102667301)
book==4 and read==9 => p(0.0), conf(0.0001097654677838352)
book==4 and read==5 and story==3 => p(0.0), conf(0.0004390618711353408)
book==4 and read==3 and story==3 => p(0.0), conf(0.0015733050382349713)
book==6 and read==5 => p(0.0), conf(0.0014269510811898577)
book==4 and read==5 and story==2 => p(0.0), conf(0.0001463539570451136)
book==4 and read==5 and story==5 => p(0.0), conf(0.000182942446306392)
book==4 and read==5 and story==6 => p(0.0), conf(3.65884892612784e-05)
book==4 and read==3 and story==2 => p(0.0), conf(0.0019757784201090336)
book==4 and read==5 and story==4 => p(0.0), conf(3.65884892612784e-05)
book==4 and read==5 and story==1 and character==6 => p(0.0), conf(7.31769785225568e-05

In [27]:
consolidar = rso.consolidar()

0
diccionary
{0: <rulesets.Rule object at 0x7f0f3041ff90>, 1: <rulesets.Rule object at 0x7f0f3041f2d0>, 2: <rulesets.Rule object at 0x7f0f3116b950>, 3: <rulesets.Rule object at 0x7f0f3bc5d210>, 4: <rulesets.Rule object at 0x7f0f304478d0>, 5: <rulesets.Rule object at 0x7f0f30447e90>, 6: <rulesets.Rule object at 0x7f0f304477d0>, 7: <rulesets.Rule object at 0x7f0f30447f90>, 8: <rulesets.Rule object at 0x7f0f30447890>, 9: <rulesets.Rule object at 0x7f0f30447b90>, 10: <rulesets.Rule object at 0x7f0f30447e10>, 11: <rulesets.Rule object at 0x7f0f30447450>, 12: <rulesets.Rule object at 0x7f0f3117e690>, 13: <rulesets.Rule object at 0x7f0f3117e090>, 14: <rulesets.Rule object at 0x7f0f305e06d0>}
visitados q no deben volver :0 -- 0 -- 1 -- 2 -- 3 -- 4 -- 5 -- 8
listado de reglas con mismo antecedente:
book==8 and read==5 => p(0.0), conf(0.0004756503603966192)
book==4 and read==10 => p(0.0), conf(0.0001463539570451136)
book==4 and read==7 => p(0.0), conf(0.0005854158281804544)
book==4 and read==8 =

In [28]:
contador = 1
for ley in consolidar.keys():
    print(consolidar.get(ley).toString())
    joblib.dump(consolidar.get(ley),'../objects/amazon/rules_2_experiment/rule_2_'+str(contador))
    contador = contador+1

operator 1>=
book==4 and ((read>=6 and read<=10)) => p(0.0), conf(0.002670959716073323)
book==6 and read==5 => p(0.0), conf(0.0014269510811898577)
book==8 and read==5 => p(0.0), conf(0.0004756503603966192)
operator 1>=
book==4 and read==5 and ((story>=5 and story<=6)) => p(0.0), conf(0.0002195309355676704)
operator 1>=
book==4 and read==5 and ((story>=3 and story<=4)) => p(0.0), conf(0.0004756503603966192)
operator 1>=
book==4 and ((story>=2 and story<=3) and read==3 or read==5) => p(0.006756756756756757), conf(0.010830192821338408)
book==4 and read==3 and story==2 => p(0.0), conf(0.0019757784201090336)
book==4 and read==5 and story==1 and character==6 => p(0.0), conf(7.31769785225568e-05)


In [72]:
## 3 Experimento: Número de reglas 10, selección de vocabulario, número máximo de repeticiones por término 2

vocabulary=['word','book','romance','story','author','character','read']
rs = RuleSet()
rs.target='target'
rs.pds=copy.deepcopy(dtm_scaled)
rs.pds_original = copy.deepcopy(dtm_scaled) 
explainer = Explainer(rs,1,10,15,3,vocabulary=vocabulary)
rs, pdsb = explainer.explain()

*****************************Comienza la genreación del ruleset************************************************
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: book
Igualdad generada: book==8POSITIVOS 23.0NEGATIVOS: 4
Igualdad de la regla book==8
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: read
Igualdad generada: read==5POSITIVOS 13.0NEGATIVOS: 0
Igualdad de la regla read==5
-----------------> COUNTER USADOS
Counter({'book': 1, 'read': 1})
|2 - Regla generada: |book==8 and read==5 => p(0.0), conf(0.0004756503603966192) 
ruleset query(book==8 and read==5) 
	 |2.1 -Quedan | 27318 positivos, generadas 2 reglas
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: book
Igualdad generada: book==4POSITIVOS 23.0NEGATIVOS: 4
Igualdad de la regla book==4
		 Buscando el atributo con ganancia maxima...
atributo con max ganancia: read
Igualdad generada: read==10POSITIVOS 4.0NEGATIVOS: 0
Igualdad de la regla read==10
------

In [73]:
rso = copy.deepcopy(rs)
for key in rso.dictionary.keys():
    print(rso.dictionary.get(key).toString())

book==8 and read==5 => p(0.0), conf(0.0004756503603966192)
book==4 and read==10 => p(0.0), conf(0.0001463539570451136)
book==4 and read==7 => p(0.0), conf(0.0005854158281804544)
book==4 and read==8 => p(0.0), conf(0.0004756503603966192)
story==9 => p(0.0), conf(0.0005122388496578977)
story==5 and character==5 => p(0.0), conf(0.0010610661885770737)
story==5 and character==6 => p(0.0), conf(0.0004756503603966192)
story==5 and character==8 => p(0.0), conf(3.65884892612784e-05)
character==6 => p(0.0), conf(0.006549339577768834)
author==8 => p(0.0), conf(0.0009147122315319601)
author==10 => p(0.0), conf(3.65884892612784e-05)
author==9 => p(0.0), conf(0.0004390618711353408)
author==6 and romance==2 => p(0.0), conf(3.65884892612784e-05)
romance==3 => p(0.0), conf(0.008195821594526363)
romance==10 => p(0.0), conf(3.65884892612784e-05)


In [74]:
consolidar = rso.consolidar()

0
diccionary
{0: <rulesets.Rule object at 0x7f0f3221d390>, 1: <rulesets.Rule object at 0x7f0f3221d4d0>, 2: <rulesets.Rule object at 0x7f0f28077590>, 3: <rulesets.Rule object at 0x7f102bdcc750>, 4: <rulesets.Rule object at 0x7f0f305f3ad0>, 5: <rulesets.Rule object at 0x7f0f305f3c50>, 6: <rulesets.Rule object at 0x7f0f305f3710>, 7: <rulesets.Rule object at 0x7f0f305f3d10>, 8: <rulesets.Rule object at 0x7f0f305d20d0>, 9: <rulesets.Rule object at 0x7f0f305d2310>, 10: <rulesets.Rule object at 0x7f0f305d28d0>, 11: <rulesets.Rule object at 0x7f0f325797d0>, 12: <rulesets.Rule object at 0x7f0f32579710>, 13: <rulesets.Rule object at 0x7f0f325792d0>, 14: <rulesets.Rule object at 0x7f0f325795d0>}
visitados q no deben volver :0 -- 0 -- 1 -- 2 -- 3
listado de reglas con mismo antecedente:
book==8 and read==5 => p(0.0), conf(0.0004756503603966192)
book==4 and read==10 => p(0.0), conf(0.0001463539570451136)
book==4 and read==7 => p(0.0), conf(0.0005854158281804544)
book==4 and read==8 => p(0.0), conf(

In [75]:
contador = 1
for ley in consolidar.keys():
    print(consolidar.get(ley).toString())
    joblib.dump(consolidar.get(ley),'../objects/amazon/rules_3_experiment/rule_3_'+str(contador))
    contador = contador+1

debería entrar aqui si tengo antecedentes, por cada disyuncion
operator 1>=
book==4 and ((read>=7 and read<=8) or 
debería entrar aqui si tengo antecedentes, por cada disyuncion
book==4 and ((read>=7 and read<=8) or read==10 or 
book==4 and ((read>=7 and read<=8) or read==10) => p(0.0), conf(0.0012074201456221873)
book==8 and read==5 => p(0.0), conf(0.0004756503603966192)
story==9 => p(0.0), conf(0.0005122388496578977)
debería entrar aqui si tengo antecedentes, por cada disyuncion
operator 1>=
story==5 and ((character>=5 and character<=6) or 
debería entrar aqui si tengo antecedentes, por cada disyuncion
story==5 and ((character>=5 and character<=6) or character==8 or 
story==5 and ((character>=5 and character<=6) or character==8) => p(0.0), conf(0.0015733050382349713)
character==6 => p(0.0), conf(0.006549339577768834)
debería entrar aqui si no tengo antecedentes
OPERATOR GREATER OR LOWER THAN
operator 1>=
((author>=8 and author<=10) ) => p(0.0), conf(0.0013903625919285793)
author==6 a